<a href="https://colab.research.google.com/github/kayiwa/SBU_Data_Mining/blob/main/francis_kayiwa_assignmentsbu_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import the useful libraries. (yank and paste from tutorial)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# this is dangerous but will do
data= pd.read_csv("https://raw.githubusercontent.com/kayiwa/SBU_Data_Mining/main/Week%202/Assignment%202.1/malware.csv")
# remove occurences of NaN in the dataset
data.drop(data.filter(regex="Unnamed"),axis=1, inplace=True)
data.drop(data.filter(regex="source_app_packets.1"),axis=1, inplace=True)
data = data.rename(columns={'vulume_bytes': 'volume_bytes'})
data = data[~data.name.isnull()].copy()
data = data[~data.tcp_packets.isnull()].copy()
data = data[~data.dist_port_tcp.isnull()].copy()
data = data[~data.volume_bytes.isnull()].copy()
data = data[~data.udp_packets.isnull()].copy()
data = data[~data.external_ips.isnull()].copy()
data = data[~data.tcp_urg_packet.isnull()].copy()
data = data[~data.source_app_packets.isnull()].copy()
data = data[~data.remote_app_packets.isnull()].copy()
data = data[~data.source_app_bytes.isnull()].copy()
data = data[~data.remote_app_bytes.isnull()].copy()
data = data[~data.dns_query_times.isnull()].copy()
data = data[~data.type.isnull()].copy()
data.isnull().sum()
# show us the data
data

,name,tcp_packets,dist_port_tcp,external_ips,volume_bytes,udp_packets,tcp_urg_packet,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times,type
0,AntiVirus,36.0,6.0,3.0,3911.0,0.0,0.0,39.0,33.0,5100.0,4140.0,3.0,benign
2,AntiVirus,196.0,0.0,6.0,24151.0,0.0,0.0,205.0,214.0,163887.0,24867.0,9.0,benign
3,AntiVirus,6.0,0.0,1.0,889.0,0.0,0.0,7.0,6.0,819.0,975.0,1.0,benign
4,AntiVirus,6.0,0.0,1.0,882.0,0.0,0.0,7.0,6.0,819.0,968.0,1.0,benign
5,AntiVirus,54.0,54.0,3.0,5062.0,0.0,0.0,63.0,54.0,5457.0,5719.0,9.0,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7835,Zitmo,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,147.0,74.0,1.0,malicious
7838,Zsone,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,257.0,143.0,2.0,malicious
7840,Zsone,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,257.0,143.0,2.0,malicious
7842,Zsone,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,257.0,143.0,2.0,malicious


In [3]:
# import preprocessing from sklearn to convert type and name to numbers that we can work with. 
from sklearn import preprocessing
label_enc = preprocessing.LabelEncoder()
data_2 = data.apply(label_enc.fit_transform)
data_2

,name,tcp_packets,dist_port_tcp,external_ips,volume_bytes,udp_packets,tcp_urg_packet,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times,type
0,5,36,6,3,950,0,0,38,33,1098,1089,3,0
2,5,195,0,6,2460,0,0,204,211,2496,2648,9,0
3,5,6,0,1,185,0,0,6,6,229,271,1,0
4,5,6,0,1,180,0,0,6,6,229,265,1,0
5,5,54,53,3,1143,0,0,62,54,1146,1341,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7835,97,0,0,0,0,0,0,0,1,27,5,1,1
7838,98,0,0,0,0,0,0,1,2,61,22,2,1
7840,98,0,0,0,0,0,0,1,2,61,22,2,1
7842,98,0,0,0,0,0,0,1,2,61,22,2,1


In [4]:
# print the column names
data_2.columns

Index(['name', 'tcp_packets', 'dist_port_tcp', 'external_ips', 'volume_bytes',
       'udp_packets', 'tcp_urg_packet', 'source_app_packets',
       'remote_app_packets', 'source_app_bytes', 'remote_app_bytes',
       'dns_query_times', 'type'],
      dtype='object')

In [6]:
# define X and Y
feature_cols = ['tcp_packets', 'external_ips', 'volume_bytes', 'source_app_packets', 'remote_app_packets', 'source_app_bytes', 'remote_app_bytes', 'dns_query_times']
X = data_2[feature_cols]
Y = data_2.type

In [7]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [8]:
# train a logistic regression model on the training set
# the fit step allows the learn the relationship between X_train and Y_train
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
# make class predictions for the testing set
y_prediction_class = logreg.predict(X_test)

In [11]:
# calculate accuracy of predictions
from sklearn import metrics
print(metrics.accuracy_score(Y_test, y_prediction_class))

0.7063166529942576



**Null accuracy:** accuracy that could be achieved by always predicting the most frequent class in the testing set. This will give us confidence in the 71% prediction. 

In [12]:
# evaluate class distribution of the testing set (in pandas we can count occurences of each value in the series)
Y_test.value_counts()

0    720
1    499
Name: type, dtype: int64

In [13]:
# calculate the percentage of ones vs zeros 
Y_test.mean()

0.40935192780968005

In [14]:
# calculate the percentage of zeros
1 - Y_test.mean()

0.59064807219032

In [15]:
# calculate null accuracy (for binary classification problems coded as 0/1)
max(Y_test.mean(), 1 - Y_test.mean())

0.59064807219032

In [16]:
# calculate null accuracy (for multi-class classification problems)
Y_test.value_counts().head(1) / len(Y_test)

0    0.590648
Name: type, dtype: float64

When we compare our 71 to the null accuracy of 59 we can say that we have a decent model. 

In [19]:
# first argument is true values, second argument is predicted values
print(metrics.confusion_matrix(Y_test, y_prediction_class))

[[587 133]
 [225 274]]


In [24]:
# print the first 25 true and predicted responses. Notice the rarity of the 1 being predicted on
print('True:', Y_test.values[0:25])
print('Pred:', y_prediction_class[0:25])

True: [0 1 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 0]
Pred: [0 1 1 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1]


In [21]:
confusion = metrics.confusion_matrix(Y_test, y_prediction_class)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [25]:
# Overall, how often is the classifier correct
print((TP + TN) / (TP + TN + FP + FN))
print(metrics.accuracy_score(Y_test, y_prediction_class))

0.7063166529942576
0.7063166529942576


In [26]:
# Overall, how often is the classifier incorrect
print((FP + FN) / (TP + TN + FP + FN))
print(1 - metrics.accuracy_score(Y_test, y_prediction_class))

0.2936833470057424
0.2936833470057424


In [29]:
# when the value is positive, how often is the prediction correct
print(TP / (TP + FN))
print(metrics.recall_score(Y_test, y_prediction_class))

0.5490981963927856
0.5490981963927856


In [30]:
# when the value negative, how often is the prediction correct
print(TN / (TN + FP))

0.8152777777777778


In [31]:
# when the value is negative, how often is the prediction incorrect
print(FP / (TN + FP))

0.18472222222222223


In [32]:
# When a positive value is predicted, how often is the prediction correct
print(TP / (TP + FP))
print(metrics.precision_score(Y_test, y_prediction_class))

0.6732186732186732
0.6732186732186732


Confusion matrix gives us a more complete picture of how your classifier is performing